In [1]:
from whoosh.qparser import *
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED,NUMERIC
from whoosh.analysis import StemmingAnalyzer,StandardAnalyzer
from whoosh import index
import os, os.path

In [2]:
schema = Schema(page=NUMERIC(stored=True),
                title=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                topic=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                abstract=TEXT(analyzer=StandardAnalyzer(stoplist=None),stored=True),
                subject=TEXT(analyzer=StandardAnalyzer(stoplist=None)),
                sub=KEYWORD(commas=True,scorable=True),
                keywords=KEYWORD(commas=True, scorable=True))

In [3]:
#to create an index in a dictionary
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = index.create_in("indexdir", schema)
#open an existing index object
ix = index.open_dir("indexdir")
#create a writer object to add documents to the index
writer = ix.writer()

In [4]:
abstract1=u'''Develop
Accelerate real-world development
Software developers and data scientists can work together with AE to build, test, debug, and deploy models using their preferred languages and tools. AE provides access to both notebooks and IDEs so developers and data scientists can work together more efficiently. They can also choose from example projects and preconfigured projects. AE projects are automatically containerized so they can be moved between environments with ease.'''


In [5]:
writer.add_document(page=u"1",
                title=u"Develop",
                topic=u"Accelerate real-world developmen ",
                abstract=abstract1,
                subject=u"Example",
                keywords=u"Software")
writer.commit()

In [6]:
parser=QueryParser("abstract",schema=schema) 
result=parser.parse(u"apple company department")
print (result)

(abstract:apple AND abstract:company AND abstract:department)


In [7]:
parser.add_plugin(GtLtPlugin()) 
result=parser.parse(u"year:>0")
print (result)

(abstract:year AND abstract:{0 TO ])


In [8]:
parser.add_plugin(FuzzyTermPlugin())
result=parser.parse(u"Develop")
print (result)
import pdb
with ix.searcher() as searcher:
    results=searcher.search(result)#The Results object acts like a list of the matched documents.
    result1 = (results[0])
    #pdb.set_trace()
    #print(result1)
    print(result1["abstract"])

abstract:develop
Develop
Accelerate real-world development
Software developers and data scientists can work together with AE to build, test, debug, and deploy models using their preferred languages and tools. AE provides access to both notebooks and IDEs so developers and data scientists can work together more efficiently. They can also choose from example projects and preconfigured projects. AE projects are automatically containerized so they can be moved between environments with ease.
